# Notebook: Stable Diffusion Forge - SD XL Turbo

## Objectif

Ce notebook pédagogique vous apprend à utiliser l'**API Stable Diffusion Forge** avec le modèle **SD XL Turbo** pour générer des images à partir de descriptions textuelles (prompts).

## Contexte

- **API**: Stable Diffusion Forge (WebUI Automatic1111)
- **Modèle**: SD XL Turbo (optimisé pour vitesse)
- **URL Base**: `https://turbo.stable-diffusion-webui-forge.myia.io`
- **Performance**: ~18s pour génération 512×512 (4 steps)

## Use Cases

- Prototypage rapide de concepts visuels
- Itération créative sur variations de prompts
- Exploration de styles artistiques
- Tests de faisabilité avant génération haute qualité

## Pré-requis

- Packages Python: `requests`, `Pillow`, `matplotlib`, `python-dotenv`
- Accès réseau à l'API Forge

In [ ]:
"""
Configuration initiale: imports et paramètres API
"""

# Imports standard
import requests
import json
import base64
from io import BytesIO
from typing import Optional, Dict, Any
import warnings

# Imports visualisation
from PIL import Image
import matplotlib.pyplot as plt

# Configuration API Forge
API_BASE_URL = "https://turbo.stable-diffusion-webui-forge.myia.io"
TIMEOUT = 60  # Timeout pour les requêtes HTTP (secondes)

# Suppression warnings non critiques
warnings.filterwarnings('ignore')

print("✅ Configuration initiale chargée")
print(f"📡 API Forge: {API_BASE_URL}")

## 1. Comprendre l'API Stable Diffusion Forge

### Architecture

L'API Forge expose une interface RESTful pour interagir avec Stable Diffusion:

- **Endpoint principal**: `/sdapi/v1/txt2img` (génération texte→image)
- **Méthode HTTP**: POST
- **Format**: JSON (requête et réponse)
- **Authentification**: Non requise (API interne)

### Paramètres Critiques SD XL Turbo

Le modèle **SD XL Turbo** est optimisé pour la **vitesse**:

| Paramètre | Valeur Optimale | Explication |
|-----------|----------------|-------------|
| `steps` | **4** | Nombre d'itérations de débruitage (vs 20-50 pour modèles standard) |
| `cfg_scale` | **2.0** | Force de guidage du prompt (vs 7.0 typiquement) |
| `sampler_name` | `DPM++ 2M` | Échantillonneur rapide compatible Turbo |
| `width` × `height` | **512×512** | Résolution par défaut pour performances optimales |

### Flux de Travail Typique

1. **Construire payload JSON** avec prompt + paramètres
2. **Envoyer requête POST** à l'endpoint `/sdapi/v1/txt2img`
3. **Recevoir réponse JSON** contenant image(s) encodée(s) en base64
4. **Décoder + afficher** image avec Pillow/Matplotlib

In [ ]:
"""
Fonction helper pour générer des images via API Forge
"""

def generate_image_forge(
    prompt: str,
    negative_prompt: str = "",
    steps: int = 4,
    cfg_scale: float = 2.0,
    width: int = 512,
    height: int = 512,
    sampler_name: str = "DPM++ 2M",
    save_path: Optional[str] = None
) -> Optional[Image.Image]:
    """
    Génère une image via l'API Stable Diffusion Forge (SD XL Turbo).
    
    Args:
        prompt: Description textuelle de l'image désirée
        negative_prompt: Éléments à éviter dans l'image
        steps: Nombre d'itérations (4 recommandé pour Turbo)
        cfg_scale: Force de guidage du prompt (2.0 recommandé pour Turbo)
        width: Largeur de l'image en pixels
        height: Hauteur de l'image en pixels
        sampler_name: Algorithme d'échantillonnage
        save_path: Chemin optionnel pour sauvegarder l'image
    
    Returns:
        Image PIL si succès, None si erreur
    """
    # Construction payload API
    payload = {
        "prompt": prompt,
        "negative_prompt": negative_prompt,
        "steps": steps,
        "cfg_scale": cfg_scale,
        "width": width,
        "height": height,
        "sampler_name": sampler_name
    }
    
    try:
        # Requête POST vers endpoint txt2img
        print(f"🎨 Génération en cours: '{prompt[:50]}...'")
        response = requests.post(
            f"{API_BASE_URL}/sdapi/v1/txt2img",
            json=payload,
            timeout=TIMEOUT
        )
        response.raise_for_status()
        
        # Extraction de l'image depuis JSON
        result = response.json()
        if "images" not in result or len(result["images"]) == 0:
            print("❌ Erreur: Aucune image générée")
            return None
        
        # Décodage base64 → PIL Image
        image_data = base64.b64decode(result["images"][0])
        image = Image.open(BytesIO(image_data))
        
        # Sauvegarde optionnelle
        if save_path:
            image.save(save_path)
            print(f"💾 Image sauvegardée: {save_path}")
        
        print(f"✅ Image générée ({width}×{height})")
        return image
        
    except requests.exceptions.Timeout:
        print(f"⏱️ Timeout après {TIMEOUT}s")
        return None
    except requests.exceptions.RequestException as e:
        print(f"❌ Erreur requête: {e}")
        return None
    except Exception as e:
        print(f"❌ Erreur inattendue: {e}")
        return None

print("✅ Fonction generate_image_forge() définie")

## 2. Exemple Simple: Première Génération

Testons la fonction avec un prompt simple pour générer un paysage de montagne au coucher du soleil.

**Note**: La génération prend environ **18 secondes** avec les paramètres Turbo optimaux (4 steps).

In [ ]:
# Génération d'un paysage de montagne au coucher du soleil
image_simple = generate_image_forge(
    prompt="A serene mountain landscape at sunset, golden hour lighting, photorealistic",
    negative_prompt="blurry, low quality, distorted",
    steps=4,
    cfg_scale=2.0,
    width=512,
    height=512
)

# Affichage du résultat
if image_simple:
    plt.figure(figsize=(8, 8))
    plt.imshow(image_simple)
    plt.axis('off')
    plt.title("Paysage de Montagne au Coucher du Soleil", fontsize=14, pad=10)
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ La génération a échoué. Vérifiez la connexion à l'API.")

## 3. Optimisation des Paramètres SD XL Turbo

### Pourquoi `steps=4` ?

SD XL Turbo utilise une **distillation de modèle** qui réduit le nombre d'étapes nécessaires :

- **Modèles standard** : 20-50 steps pour qualité optimale
- **SD XL Turbo** : **4 steps** suffisent grâce à l'entraînement spécialisé

### Pourquoi `cfg_scale=2.0` ?

Le **Classifier-Free Guidance (CFG)** contrôle l'adhérence au prompt :

| Valeur | Effet | Usage |
|--------|-------|-------|
| 1.0 | Créativité maximale, prompt ignoré | Exploration aléatoire |
| **2.0** | **Équilibre Turbo optimal** | **Génération rapide cohérente** |
| 7.0 | Adhérence stricte (standard) | Modèles non-Turbo |
| 15.0+ | Sur-saturation, artefacts | À éviter |

### Samplers Compatibles

Samplers recommandés pour SD XL Turbo :

- **DPM++ 2M** (par défaut) : Rapide et stable
- **Euler a** : Variations créatives
- **LMS** : Qualité légèrement supérieure

⚠️ **Éviter** : DDIM, PLMS (trop lents pour Turbo)

In [ ]:
# Test des paramètres optimaux Turbo
print("🧪 Test comparatif: Impact du nombre de steps\n")

prompt_test = "A futuristic city at night, neon lights, cyberpunk style"

# Test avec 4 steps (optimal Turbo)
print("1️⃣ Génération avec 4 steps (Turbo optimal)...")
image_turbo = generate_image_forge(
    prompt=prompt_test,
    steps=4,
    cfg_scale=2.0
)

# Note: Pour comparer avec 20 steps (standard), décommentez ci-dessous
# print("\n2️⃣ Génération avec 20 steps (standard)...")
# image_standard = generate_image_forge(
#     prompt=prompt_test,
#     steps=20,
#     cfg_scale=7.0
# )

if image_turbo:
    plt.figure(figsize=(8, 8))
    plt.imshow(image_turbo)
    plt.axis('off')
    plt.title("Ville Futuriste (4 steps, cfg=2.0)", fontsize=14)
    plt.tight_layout()
    plt.show()
    
print("\n✅ Résultat: 4 steps suffisent pour qualité acceptable en ~18s")

## 4. Cas d'Usage Avancé: Comparaison de Prompts

Explorons comment de **légères variations de prompt** influencent le résultat final.

### Technique: Grid Comparison

Génération de plusieurs images avec prompts légèrement différents pour identifier les formulations les plus efficaces.

In [ ]:
# Définition de 3 variations de prompt
prompts_comparison = [
    "A futuristic city at night",
    "A futuristic city at night, neon lights, rainy streets",
    "A futuristic city at night, cyberpunk style, flying cars, neon signs"
]

print("🎨 Génération de 3 variations de prompt...\n")

# Génération des images
images_grid = []
for i, prompt in enumerate(prompts_comparison, 1):
    print(f"{i}. \"{prompt}\"")
    img = generate_image_forge(
        prompt=prompt,
        negative_prompt="blurry, low quality",
        steps=4,
        cfg_scale=2.0
    )
    if img:
        images_grid.append(img)
    print()

# Affichage en grille
if len(images_grid) == 3:
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    for idx, (ax, img, prompt) in enumerate(zip(axes, images_grid, prompts_comparison)):
        ax.imshow(img)
        ax.axis('off')
        # Titre tronqué pour affichage
        short_title = prompt if len(prompt) <= 40 else prompt[:37] + "..."
        ax.set_title(f"Variation {idx+1}\n{short_title}", fontsize=10)
    
    plt.suptitle("Comparaison: Impact des Variations de Prompt", fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Analyse:")
    print("- Prompt simple → Interprétation générique")
    print("- Prompt + détails → Contrôle accru sur atmosphère")
    print("- Prompt + style → Cohérence artistique maximale")
else:
    print("⚠️ Certaines générations ont échoué")

## 5. Bonnes Pratiques et Recommandations

### ✅ À Faire

1. **Prompts descriptifs**: Inclure style, éclairage, qualité
   ```
   "A serene lake, sunset lighting, photorealistic, 8k quality"
   ```

2. **Negative prompts efficaces**: Spécifier défauts courants
   ```
   "blurry, low quality, distorted, watermark, signature"
   ```

3. **Paramètres Turbo**: Toujours utiliser `steps=4` et `cfg_scale=2.0`

4. **Résolution optimale**: 512×512 pour vitesse maximale

5. **Gestion erreurs**: Vérifier `image is not None` avant utilisation

### ❌ À Éviter

1. **Steps élevés**: > 4 steps annule l'avantage Turbo
2. **CFG élevé**: `cfg_scale > 3.0` produit artefacts avec Turbo
3. **Résolutions extrêmes**: < 256×256 ou > 768×768 (dégradation qualité)
4. **Prompts vagues**: "une image" → résultats imprévisibles
5. **Timeouts courts**: Génération prend ~18s, timeout < 30s risqué

### 🎯 Cas d'Usage Recommandés

| Scénario | SD XL Turbo | Autres Modèles |
|----------|-------------|----------------|
| Prototypage rapide | ✅ Idéal | ❌ Trop lent |
| Itération créative | ✅ Excellent | ⚠️ Acceptable |
| Production finale | ⚠️ Acceptable | ✅ Préférable |
| Haute résolution (>1024px) | ❌ Non adapté | ✅ Recommandé |

In [ ]:
"""
Exemple 4: Version améliorée avec logging coloré
(Inspiré du pattern LocalLlama)
"""

from enum import Enum

class LogColor(Enum):
    """Codes ANSI pour logging coloré dans notebooks"""
    RESET = '\033[0m'
    INFO = '\033[94m'      # Bleu
    SUCCESS = '\033[92m'   # Vert
    WARNING = '\033[93m'   # Jaune
    ERROR = '\033[91m'     # Rouge
    HEADER = '\033[95m'    # Magenta

def log_colored(message: str, color: LogColor = LogColor.INFO):
    """Affiche message coloré"""
    print(f"{color.value}{message}{LogColor.RESET.value}")

def generate_image_forge_v2(
    prompt: str,
    verbose: bool = True,
    **kwargs
) -> Optional[Image.Image]:
    """
    Version améliorée avec logging détaillé
    """
    if verbose:
        log_colored(f"🎨 Prompt: '{prompt}'", LogColor.HEADER)
        log_colored(f"⚙️  Paramètres: steps={kwargs.get('steps', 4)}, "
                   f"size={kwargs.get('width', 512)}×{kwargs.get('height', 512)}",
                   LogColor.INFO)
    
    # Génération
    image = generate_image_forge(prompt=prompt, **kwargs)
    
    # Logging résultat
    if image:
        if verbose:
            log_colored(f"✅ Génération réussie: {image.size[0]}×{image.size[1]}", 
                       LogColor.SUCCESS)
        return image
    else:
        if verbose:
            log_colored("❌ Échec génération", LogColor.ERROR)
        return None

# Test
print("Test fonction avec logging coloré:")
print("-" * 50)
img_test = generate_image_forge_v2(
    prompt="A cozy cabin in snowy mountains",
    steps=4,
    width=512,
    height=512,
    verbose=True
)

if img_test:
    plt.figure(figsize=(6, 6))
    plt.imshow(img_test)
    plt.axis('off')
    plt.title("Test Logging Coloré", fontsize=11)
    plt.show()

## 6. Exercice Pratique

### Objectif

Créez votre propre génération d'image en appliquant les techniques apprises.

### Instructions

1. **Choisissez un sujet**: Paysage, portrait, objet, scène abstraite, etc.
2. **Rédigez un prompt descriptif**: Incluez style, éclairage, qualité
3. **Définissez un negative prompt**: Spécifiez les défauts à éviter
4. **Utilisez les paramètres Turbo optimaux**: `steps=4`, `cfg_scale=2.0`
5. **Générez et affichez l'image**

### Template de Code

Complétez le code ci-dessous avec vos propres paramètres :

In [ ]:
# 🎯 EXERCICE PRATIQUE: Complétez ce code avec vos propres paramètres

# TODO: Définissez votre prompt personnalisé
mon_prompt = "Votre description ici"  # Ex: "A majestic dragon flying over a medieval castle"

# TODO: Définissez votre negative prompt
mon_negative_prompt = ""  # Ex: "blurry, low quality, distorted"

# Génération de votre image
mon_image = generate_image_forge(
    prompt=mon_prompt,
    negative_prompt=mon_negative_prompt,
    steps=4,          # Paramètre Turbo optimal
    cfg_scale=2.0,    # Paramètre Turbo optimal
    width=512,
    height=512
)

# Affichage du résultat
if mon_image:
    plt.figure(figsize=(8, 8))
    plt.imshow(mon_image)
    plt.axis('off')
    plt.title(f"Ma Création\n{mon_prompt[:60]}...", fontsize=12)
    plt.tight_layout()
    plt.show()
    
    print("\n✅ Félicitations! Votre image a été générée avec succès.")
    print(f"📝 Prompt utilisé: {mon_prompt}")
else:
    print("❌ La génération a échoué. Vérifiez vos paramètres et la connexion à l'API.")

## 7. Ressources et Documentation

### Documentation Complète

Pour approfondir vos connaissances sur l'API Forge et SD XL Turbo, consultez :

- **Guide Étudiants API Forge**: [`docs/suivis/genai-image/GUIDE-APIS-ETUDIANTS.md`](../../../docs/suivis/genai-image/GUIDE-APIS-ETUDIANTS.md)
- **Documentation Stable Diffusion**: [Stability AI Documentation](https://platform.stability.ai/docs)
- **API Forge WebUI**: [Automatic1111 API Wiki](https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/API)

### Endpoints API Disponibles

| Endpoint | Description | Méthode |
|----------|-------------|--------|
| `/sdapi/v1/txt2img` | Génération texte→image | POST |
| `/sdapi/v1/img2img` | Transformation d'image | POST |
| `/sdapi/v1/options` | Configuration API | GET/POST |
| `/sdapi/v1/sd-models` | Liste modèles disponibles | GET |

### Paramètres Avancés (Optionnels)

Explorez ces paramètres supplémentaires pour contrôle accru :

```python
{
    "seed": -1,              # -1 = aléatoire, valeur fixe = reproductible
    "batch_size": 1,        # Nombre d'images générées simultanément
    "restore_faces": false, # Amélioration automatique des visages
    "tiling": false,        # Image répétable (textures)
    "enable_hr": false      # Hires.fix (haute résolution)
}
```

### Support et Contact

Pour toute question ou problème technique :

- **API Status**: Vérifiez l'accessibilité à `https://turbo.stable-diffusion-webui-forge.myia.io`
- **Documentation Projet**: Consultez le README principal du projet CoursIA
- **Issues GitHub**: Signalez bugs/améliorations sur le dépôt du projet